Based on [mlopszoomcamp-chapter2-Model Registry](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/02-experiment-tracking/model-registry.ipynb)

In [ ]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///../experiments/mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [23]:
client.search_experiments()

[<Experiment: artifact_location='/mnt/d/projects/mlops_example/nb/mlruns/2', creation_time=1718586191452, experiment_id='2', last_update_time=1718586191452, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/mnt/d/projects/mlops_example/nb/mlruns/1', creation_time=1718519409655, experiment_id='1', last_update_time=1718519409655, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1718517402268, experiment_id='0', last_update_time=1718517402268, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
client.create_experiment(name="my-cool-experiment")

'2'

In [34]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.mae < 40",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=40,
    order_by = ["metrics.mae ASC"]

)

In [35]:
for run in runs:
    print(f"run id: {run.info.run_id}, mae:{run.data.metrics['mae']:.4f}")

run id: aedd49e2a0064a7cb4b8a68f6e671964, mae:4.1601
run id: 3abf44d126184d95acee1f6dd68f2e06, mae:4.1601
run id: e624f0cd9088417abd21f8056835d457, mae:4.1763
run id: 0652498fa132463696388baacfd2cac8, mae:4.2104
run id: 15df42e1b53e43b8b219b5277a7d58db, mae:4.2359
run id: e1fa557e8e934907b63d44906afbebc8, mae:4.2625
run id: 8c062bec68a241ceb240f0539ae592a8, mae:4.2663
run id: d29afa606bc74723b9cbfdfb6f5cce88, mae:4.2692
run id: ecb786aa4b9441c0affc9de7baff9838, mae:4.3262
run id: be35763aed9243edba1208a95389ab35, mae:4.3263
run id: 8f0166c41e1244c29bbff38c1a2c07b2, mae:4.3519
run id: 6b1c8076d1de41a6bec1cc840f14dfdc, mae:4.3530
run id: 902171bccb6b47169aeb8c85d64988f6, mae:4.4012
run id: 7c40fff50d4445a2a1b9c47198bed061, mae:4.5184
run id: ee78443185f6475babc5d755c69b13f1, mae:4.6035
run id: 0d4effe279b24a1da71f7d379621d428, mae:5.2412
run id: d94e0a431cdb4302a5beb5bc470c5650, mae:8.2296
run id: 42c0dafbdaac45ddafdd67b3c18531d1, mae:9.3309
run id: ca7ac5269ab84c68a7c3b7733efcaecb, mae:

In [14]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [36]:
# Create a new version of the model to REGISTER MODEL

run_id = "ee78443185f6475babc5d755c69b13f1"

model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor-short")

Registered model 'nyc-taxi-regressor-short' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor-short'.


<ModelVersion: aliases=[], creation_timestamp=1718598841471, current_stage='None', description=None, last_updated_timestamp=1718598841471, name='nyc-taxi-regressor-short', run_id='ee78443185f6475babc5d755c69b13f1', run_link=None, source='/mnt/d/projects/mlops_example/nb/mlruns/1/ee78443185f6475babc5d755c69b13f1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [37]:
# Transition a model to another stage

model_name = "nyc-taxi-regressor-short"
client.get_latest_versions(name=model_name)

/tmp/ipykernel_29168/4177189500.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1718597910653, current_stage='Staging', description='', last_updated_timestamp=1718597963283, name='nyc-taxi-regressor-short', run_id='0d4effe279b24a1da71f7d379621d428', run_link='', source='/mnt/d/projects/mlops_example/nb/mlruns/1/0d4effe279b24a1da71f7d379621d428/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>,
 <ModelVersion: aliases=[], creation_timestamp=1718597923962, current_stage='Production', description='', last_updated_timestamp=1718597953431, name='nyc-taxi-regressor-short', run_id='ce49d85579c24f3ab226935b8289494a', run_link='', source='/mnt/d/projects/mlops_example/nb/mlruns/1/ce49d85579c24f3ab226935b8289494a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>,
 <ModelVersion: aliases=[], creation_timestamp=1718598841471, current_stage='None', description=None, last_updated_timestamp=1718598841471, name='nyc-taxi-regressor-short', run_id='ee78443185

In [38]:
model_name = "nyc-taxi-regressor-short"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: Production
version: 4, stage: None


/tmp/ipykernel_29168/1315063750.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [39]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage='Staging',
    archive_existing_versions=False
)

/tmp/ipykernel_29168/3888885130.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1718598841471, current_stage='Staging', description=None, last_updated_timestamp=1718598893971, name='nyc-taxi-regressor-short', run_id='ee78443185f6475babc5d755c69b13f1', run_link=None, source='/mnt/d/projects/mlops_example/nb/mlruns/1/ee78443185f6475babc5d755c69b13f1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [40]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1718597886168, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-06-16', last_updated_timestamp=1718598926274, name='nyc-taxi-regressor-short', run_id='ee78443185f6475babc5d755c69b13f1', run_link='', source='/mnt/d/projects/mlops_example/nb/mlruns/1/ee78443185f6475babc5d755c69b13f1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>